In [1]:
import time, os, threading, cv2
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def scrape(path, headless=False):
    chrome_options = ChromeOptions()
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--hide-scrollbars")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(path)
    driver.maximize_window()
    
    # Login Process
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 

    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
            check.click()
    except Exception as e:
        print(f"No check tab found and the error is {e}")
        
    try:
        popup = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@id="start-btn"]')))
        if popup:
            popup.click()
    except Exception as e:
        print("No popup found")
        
        
    return driver

# def record_website_screen(driver, output_file="live_rec.avi", fps=8):
#     try:
#         img = driver.get_screenshot_as_png()
#         nparr = np.frombuffer(img, np.uint8)
#         frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#         height, width, _ = frame.shape
#         fourcc = cv2.VideoWriter_fourcc(*'XVID')
#         out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

#         frame_count = 0
#         while True:
#             img = driver.get_screenshot_as_png()
#             nparr = np.frombuffer(img, np.uint8)
#             frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
#             frame_count += 1
#             out.write(frame)
#             time.sleep(1/fps)
#     except Exception as e:
#         print(f"Error in screen recording: {e}")
#     finally:
#         out.release()
#         print(f"Recording saved, captured {frame_count} frames.")

# def start_recording(driver):
#     thread = threading.Thread(target=record_website_screen, args=(driver,))
#     thread.daemon = True
#     thread.start()
def save_full_screenshot(driver, details):
    try:
        os.makedirs("screenshots", exist_ok=True)
        reg_number = details.get("Registration", "reg").replace(" ", "_")
        filename = f"screenshots/{reg_number}.png"
        driver.save_screenshot(filename)
        print(f"📸 Screenshot saved: {filename}")
    except Exception as e:
        print(f"❌ Screenshot failed: {e}")
        
        
if __name__ == "__main__":
    path = "https://www.cityauctiongroup.com/portal/auction/buyer/webauction/auction/1916"
    driver = scrape(path, headless=False)
    print("Starting screen recording...")
    # start_recording(driver)
    
    print("Starting data extraction...")
    
    
    results = []
    previous_title = ""

    try:
        while True:
            if not driver.window_handles:
                print("Browser window closed. Stopping script.")
                break
            details = {}
            try:
                # Extract Vehicle Title
                car_title =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-9"]/h1')))
                current_title = car_title.text.strip()

                # If title changes, reset Bidding History
                if current_title != previous_title:
                    previous_title = current_title  # Update last seen title
                    details["Title"] = current_title
                    
                    try:
                        for index in [1, 2]:
                            body =WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'(.//table[@class="table table-condensed vehicle-table"])[{index}]/tbody')))
                            if body:
                                trs = body.find_elements(By.TAG_NAME, 'tr')
                                for tr in trs:
                                    lbl = tr.find_element(By.TAG_NAME, 'th').text.strip()
                                    value=  tr.find_element(By.TAG_NAME, 'td').text.strip()
                                    if lbl  and value:
                                        details[lbl] = value
                                        
                                        
                            
                    except:
                        pass 
                    # Extract Bidding History
                    bidding_prices = driver.find_elements(By.XPATH, '//ul[@id="biddinghistory"]/li')
                    details["Bid History"] = [price.text.strip() for price in bidding_prices]
                    save_full_screenshot(driver, details)

                    results.append(details)

                    # Save dynamically after each successful capture
                    df = pd.DataFrame(results)
                    df.to_csv("live_data.csv", index=False)

                time.sleep(2)  # Wait before checking for updates
                
            except Exception as e:
                print("No more cars or error occurred:", e)
                    
    except KeyboardInterrupt:
        print("Stopped by user.")
    finally:
        driver.quit()
        print("Browser closed.")


Starting screen recording...
Starting data extraction...
📸 Screenshot saved: screenshots/PGZ8170.png
📸 Screenshot saved: screenshots/RGZ9747.png
📸 Screenshot saved: screenshots/DS70XDZ.png
📸 Screenshot saved: screenshots/RGZ9751.png
📸 Screenshot saved: screenshots/FSZ4358.png
📸 Screenshot saved: screenshots/KP63JZC.png
📸 Screenshot saved: screenshots/CSZ2954.png
📸 Screenshot saved: screenshots/SW67NAE.png
📸 Screenshot saved: screenshots/KN65OTX.png
📸 Screenshot saved: screenshots/FE19SVN.png
📸 Screenshot saved: screenshots/NGZ1082.png
Browser closed.


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x60fe43+66515]
	GetHandleVerifier [0x0x60fe84+66580]
	(No symbol) [0x0x3fdc48]
	(No symbol) [0x0x3eb9a4]
	(No symbol) [0x0x3ec937]
	(No symbol) [0x0x3fe6a3]
	(No symbol) [0x0x3dbcf5]
	(No symbol) [0x0x47a708]
	(No symbol) [0x0x46b193]
	(No symbol) [0x0x43c2ca]
	(No symbol) [0x0x43d154]
	GetHandleVerifier [0x0x867353+2521315]
	GetHandleVerifier [0x0x8622d3+2500707]
	GetHandleVerifier [0x0x637c94+229924]
	GetHandleVerifier [0x0x6281f8+165768]
	GetHandleVerifier [0x0x62ecad+193085]
	GetHandleVerifier [0x0x618158+100072]
	GetHandleVerifier [0x0x6182f0+100480]
	GetHandleVerifier [0x0x6025aa+11066]
	BaseThreadInitThunk 

In [4]:
top_car_df = pd.read_csv("live_data.csv")
top_car_live_cleaned_df = pd.read_csv("cag_data.csv")

# Rename 'Name' to 'Title' in Top_car dataset to match
top_car_df.rename(columns={'Registration': 'Reg'}, inplace=True)

merged_df = top_car_live_cleaned_df.merge(top_car_df[['Reg',"Bid History"]], on='Reg', how='left')
merged_df= merged_df[merged_df['Bid History'].notna()]
# Save the updated dataset
output_path = "cag_data_merged_final.csv"
merged_df.to_csv(output_path, index=False)

# df = pd.read_csv("cag_data_merged_final.csv")
# # df = df.sort_values(by='Make', ascending=True)
# df.to_csv("Final_CAG.csv", index=False)

# os.remove("cag_data.csv")
# os.remove("live_data.csv")
# os.remove("cag_data_merged_final.csv")

In [5]:
df = pd.read_csv("cag_data_merged_final.csv")
df['Lot'] = df['Lot'].astype(int)
r = df['Bid History'].iloc[-1].split("[")[1].split("]")[0]
parts = r.split("Lot ")[1:]
parts = [e for e in parts if "changed:" not in e]
all_lot_info = []
for p in parts[1:]:
    lot_num = int(p.split()[0])
    bid_status = p.split()[1]
    bid_hist = p.split()[1:]
    bids = " ".join(bid_hist)
    all_lot_info.append({"Lot": lot_num, "Bidding History": bids, "Bidding status": bid_status})
lot_df = pd.DataFrame(all_lot_info)
lot_df['Last Bid'] = lot_df['Bidding History'].apply(lambda x: x.split("£")[1].split("'")[0] if "£" in x else None)
final_df = pd.merge(df, lot_df, on="Lot", how="left")
final_df.to_csv("Final_CAG.csv", index=False)

IndexError: single positional indexer is out-of-bounds